# [You Don't Know JavaScript: this & Object Prototypes](https://www.oreilly.com/library/view/you-dont-know/9781491905142/)
Kyle Simpson (O'Reilly)  
Copyright 2014 Getify Solutions Inc.  
978-1-491-90415-2

___

## this or That?
Chapter 1, Page 1 - 9
    
<code>this</code> is a special identifier keyword that's automatically defined in the scope of every function, but it doesn't refer to itself, nor does it refer to its scope.

**Incorrect Assumptions**
1. **Itself** - assuming <code>this</code> refers to the function itself
2. **Its Scope** - assuming <code>this</code> refers to the function scope 

**Note:** The following examples are explicitly demonstrations on the incorrect assumptions about <code>this</code>. They do not illistrate on how to use <code>this</code>. That will be demonstrated later on.

### 1. Itself
<code>this</code> doesn't let a function get a reference to itself. 

In [1]:
// DONT DO THIS
// Example of how `this` doesn't refer to itself
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        this.count++;
    }
    
    foo.count = 0; // Accidently creates a global variable with a NaN value
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo(i);
        }
    }
    
    console.log(foo.count); // 0?
    
})();

foo: 6
foo: 7
foo: 8
foo: 9
0


A lack of understanding what <code>this</code> means and how it works; developers often fall back to a comfort zone by storing state into lexical scope (like objects). There are better places to store state besides the function object. You'll come to find that passing these state objects as an explicit parameter is often messier than passing around a <code>this</code> context. For example:

In [2]:
// DONT DO THIS
// Example of developers avoid `this` and using an object to store state
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        data.count++;
    }
    
    let data = { // Object storing state
        count: 0
    };
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo(i);
        }
    }
    
    console.log(data.count); // 4
    
})();

foo: 6
foo: 7
foo: 8
foo: 9
4


Another way developers avoid <code>this</code> is to reference a function object from inside itself via a lexical identifier (variable) that points at it.

In [3]:
// DONT DO THIS
// Example of using a lexical identifier that points to itself
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        foo.count++; // The lexical identifier
    }
    
    foo.count = 0; // No longer creates a global variable
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo(i);
        }
    }
    
    console.log(foo.count); // 4

})();

foo: 6
foo: 7
foo: 8
foo: 9
4


Another way developers use <code>this</code> incorrectly is to force <code>this</code> to point to the function object

In [4]:
// DONT DO THIS
// Example of forcing `this` to point to a function object
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        this.count++;
    }
    
    foo.count = 0; // No longer creates a global variable
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo.call(foo, i); // Forcing `this` to point to `foo` by using call()
        }
    }
    
    console.log(foo.count); // 4

})();

foo: 6
foo: 7
foo: 8
foo: 9
4


### 2. Its Scope
<code>this</code> does not, in any way, refer to a function's lexical scope. It is true that internally, scope is kind of like an object with properties for each of the available identifiers. But the scope "object" is not accessible to JavaScript code. It's an inner part of the engine's implementation.

In [5]:
// Example of trying to refer to a functions lexical scope
(()=>{

    function foo() {
        var a = 2;
        this.bar();
    }
    
    function bar() {
        console.log(this.a);
    }
    
    foo();

})();

TypeError: this.bar is not a function

You cannot use a <code>this</code> reference to look something up in a lexical scope. Everytime you feel yourself trying to mix lexical scope look-ups with <code>this</code>, remind yourself: there is no bridge.

### What's <code>this</code>?
<code>this</code> is not an author-time binding, but a runtime binding. <code>this</code> has nothing to do with where a function is declared, but has instead everything to do with the manner in which the function was called. When a function is invoked, an activation record, otherwise known as an execution context, is created. This record contains information about where the function was called from (the call stack), how the function was invoked, what parameters were passed etc. One of the properties of this record is the <code>this</code> reference, which will be used for the duration of that functions execution. <code>this</code> is actually a binding that is made when a function is invoked, and what it references is determined entirely by the call-site where the function is called.

___
## <code>this</code> All Makes Sense Now!
Chapter 2, Page 11 - 34
    
### Call-Site
**Call-Site** is the location in code where a function is called (not where it's declared). Finding the call-site is generally "go locate where a function is called from", but it's not always that easy, as certain coding patterns can obscure the true call-site. The call-site we care about is in the invocation before the currently executing function.

In [6]:
(() => {

    function baz() {
        // call-stack is `baz`
        // call-site is global scope
        console.log('baz');
        bar(); // call-site for `bar`
    }
    
    function bar() {
        // call-stack is `baz` --> `bar`
        // call-site is in `baz`
        console.log('bar');
        foo(); // call-site for `foo`
    }
    
    function foo() {
        // call-stack is `baz` --> `bar` --> `foo`
        // call-site is in `bar`
        console.log('foo');
    }
    
    baz(); // call-site for `baz`
    
})();

baz
bar
foo


**Special Note**: take care when analyzing code to find the actual call-site (from the call-stack), because it's the only thing that matters for <code>this</code> binding. Another way to view the call stack is through a debugger in your browser.

### Rules
There are four rules that determines where <code>this</code> will point during the execution of a function. You must inspect the call-site and determine which of the four rules applies. Multiple rules could apply to a call stack.

**Remember**: <code>this</code> only points to the direct object on how it is being called, not to whom it belongs too. Only the top/last level of an object property reference chain matters to the call-site.

#### 1. Default Binding
The default binding is the global context.<code>this</code> points the the global context, but only if there are no other rules are applied. 

In [7]:
// Default binding is the global context
(() => {
    
    a = 2; // global variable

    function foo() {
        console.log(this.a);
    }

    foo();
    
    delete a;
    
})();

2


**Special Note:** strict mode will set the global context <code>undefined</code>

In [8]:
// Strict Mode Global Context `undefined`
// A global variable doesn't need to be instantiated for this demonstration to work
(() => {
    'use strict';
        
    function foo(){
        console.log(this.a);
    }

    foo();
})();

TypeError: Cannot read properties of undefined (reading 'a')

#### 2. Implicit Binding
A context object, otherwise known as a owning object, or containing object, is an object who can either own or reference a property or value. Only the top/last level of an object property reference chain matters to the call-site.

In [9]:
// Implicit binding example
(() => {

    function foo() {
        console.log(this.a);
    }
    
    let obj2 = {
        a: 42,
        foo: foo
    };
    
    let obj1 = {
        a: 2,
        obj2: obj2,
        foo: foo
    };
    
    // Implicit Binding
    // obj2 is the last level object
    // foo points to obj2
    obj1.obj2.foo(); // 42
    
    // Implicit Binding
    // obj1 is the last level object
    // foo points to obj1
    obj1.foo(); // 2
    
    // Default Binding
    // global namespace is the last level object
    // foo points to global
    foo(); // undefined

})();

42
2
undefined


#### 3. Explicit Binding
Is the process of **forcing** <code>this</code> to reference a particular object.

The vast majority of functions, even the ones you create, contain additional function properties that are prototyped automatically. These functions take as their first parameter, and object to use for <code>this</code>.

* <code>[[Prototype]].call()</code>
* <code>[[Prototype]].apply()</code>

In [10]:
// Explicit Binding
(() => {
    
    function foo() {
        console.log(this.a);
    }
    
    let obj = {
        a: 2
    };
    
    foo.call(obj); // 2
    foo.apply(obj); // 2
    
})();

2
2


#### <code>call()</code> vs <code>apply()</code>
* <code>call()</code> will take a reference functions arguments as separate arguments
* <code>apply()</code> will take a reference functions arguments as an array argument

In [11]:
// call() vs apply()
(() => {
    
    function foo(num2) {
        console.log(this.num1 + num2);
    }
    
    let obj1 = {
        num1: 10
    };
    
    foo.call(obj1, 10); // 20
    foo.apply(obj1, [20]); // 30
    
})();

20
30


**Boxing** is the process of passing a primitive value (<code>string</code>, <code>boolean</code>, <code>number</code>) as a <code>this</code> binding for <code>call()</code> or <code>apply()</code>, it is often wrapped with its object form (<code>new String()</code>, <code>new Boolean()</code>, <code>new Number()</code>)

**Hard Binding** is the process of using explicit binding inside of a function that will never lose its <code>this</code> reference. This technique can be done manually, or through a built in ES5 utility called <code>bind</code>

In [12]:
// Manual Hard Binding
(() => {

    function foo() {
        console.log(this.a);
    }
    
    let obj = {
        a: 2
    };
    
    let bar = () => {
        foo.call(obj); // Hard binded
    };
    
    bar(); // 2
    setTimeout(bar, 100); // 2
    
})();

2


In [13]:
(() => {

    function foo() {
        console.log(this.a);
    }
    
    let obj = {
        a: 2
    };
    
    let bar = foo.bind( obj ); // Hard binded
    
    bar(); // 2

})();

2


#### 4. <code>new</code> Binding
JavaScript has a <code>new</code> operator, but **JavaScript has no connection to class oriented functionality**. The <code>new</code> operator just fires a function. In JavaScript, constructors are just functions that happen to be called with the <code>new</code> operator in front of them. They are not attached to classes, nor are they instantiating a class. They are not even special types of function.

When a function is invoked with a <code>new</code> in front of it, "construtor call", the following things are done automatically:
1. A brand new object is created
2. Newly constructor object is [[Prototype]] linked
3. Newly constructed object is set as the <code>this</code> binding for that function call
4. Unless the function returns its own alternate object, the new invoked function call will automotically return the newly constructed object

In [14]:
// new Binding Demonstration
(() => {

    function foo(a){
        this.a = a;
    }
    
    let bar = new foo(2); // binds `foo` as `this`
    
    console.log(bar.a);
    
})();

2


#### Binding Precedence
This list starting from the top, is lowest precedence to highest precedence

* Default Binding (lowest priority)
* Implicit Binding
* Explicit Binding, <code>new</code> Binding

**Special Note** <code>new</code> and <code>call()</code>/<code>apply()</code> (A.K.A Explicit Binding) cannot be used together. Explicit binding creates a new wrapper function that is hardcoded to ignore its own <code>this</code> binding, and use a manual one we provide.

In [15]:
// Demonstrating new Binding and Explicit Binding precedence
(() => {

    function foo(something) {
        this.a = something;
    }
    
    let obj = {};
    
    // Explicit Binding
    let bar = foo.bind(obj);
    bar(2);
    console.log(obj.a); // 2
    
    // new Binding
    let baz = new bar(3); // creates a completely different object for binding
    console.log(obj.a); // 2
    console.log(baz.a); // 3

})();

2
2
3


#### Determine <code>this</code>
Now that each binding method and precedence has been demonstrated. Here are a few quick rules for determining what <code>this</code> is from a call-stack.

1. Is the function called with <code>new</code>? IF so, <code>this</code> is the newly constructed object.
2. Is the function called with <code>call()</code> or <code>apply()</code>, even hidden inside a <code>bind()</code>? IF so, <code>this</code> is the explicitly specified object.
3. Is the function called with a context (implicit binding), ortherwise known as an owning or containing object? If so, <code>this</code> is that context object.
4. Otherwise, default the <code>this</code> to the global object. If strict mode is on, <code>this</code> points to undefined

### Lexical <code>this</code>
Normal functions abide by the four rules, but the ES6 fat arrow <code>=\></code> does not. The fat arrow functions adopt <code>this</code> from the enclosing (function or global) scope. The lexical binding of an arrow-function cannot be overridden (even with <code>new</code>!). Arrow functions provide an alternative to using <code>bind()</code>, but essentially disables the traditional <code>this</code> mechanism in favor to lexical scoping.

In [16]:
// Fat Arrow Lexical demonstration
(() => {

    function foo() {
        // return an arrow function
        return (a) => {
            // `this` here is lexically inherited from `foo()`
            console.log(this.a);
        };
    }
    
    let obj1 = {
        a: 2
    };
    
    let obj2 = {
        a: 3
    };
    
    let bar = foo.call(obj1);
    bar.call(obj2); // 2, not 3!

})();

2


___


In [ ]:
// continue on page 35